In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
import time
import csv
from huggingface_hub import notebook_login
import pandas as pd
from IPython.display import clear_output
import sacrebleu
from bert_score import score


[2025-04-14 21:55:40,429] [WARNING] [real_accelerator.py:194:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2025-04-14 21:55:40,434] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cpu (auto detect)


W0414 21:55:41.124699 5532 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [3]:
notebook_login()

In [ ]:
# Load legal_t5 tokenizer and model

# Replace with the model being used
model_name = "legal_t5"
model_id = "SEBIS/legal_t5_small_trans_fr_en"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
translation_pipeline = pipeline("translation_fr_to_en", model=model, tokenizer=tokenizer, device=0)

Device set to use cpu


In [6]:
# Open and read the .fr file
with open("new_dataset/test.fr", "r", encoding="utf-8") as fr_file:
    english_test_df = fr_file.readlines()

# Open and read the .en file
with open("new_dataset/test.en", "r", encoding="utf-8") as en_file:
    french_test_df = en_file.readlines()

# Combine into a list of dictionaries
test_features = [{"input": fr.strip(), "reference": en.strip()} for fr, en in zip(french_test_df, english_test_df)]

# Print a few examples
print(test_features[:5])  # Display the first 5 dictionaries

[{'input': 'Tel: +31 (0)182 557 755', 'reference': 'Tel: +31 (0)182 557 755'}, {'input': '5.3 Données de sécurité précliniques Les données précliniques issues des études conventionnelles de pharmacologie de sécurité, toxicologie en administration répétée, génotoxicité, cancérogenèse, n’ ont pas révélé de risque particulier pour l’ homme.', 'reference': '5.3 Preclinical safety data Preclinical data revela no special hazard for humans based on conventional studies of safety pharmacology, repeated dose toxicity, genotoxicity and carcinogenic potential.'}, {'input': 'Le contenu de la seringue pré-remplie doit être secoué, puis administré par une injection sous-cutanée (sous la peau).', 'reference': 'The contents of the prefilled syringe are shaken and given by subcutaneous (under the skin) injection.'}, {'input': 'Si vous avez pris plus de Sebivo que vous n’ auriez dû Si vous avez pris trop de Sebivo ou si quelqu’ un d’ autre a pris votre solution buvable par erreur, consultez immédiatemen

In [7]:
print(test_features[930:940])

[{'input': '15.', 'reference': '15.'}, {'input': 'Autriche', 'reference': 'Austria'}, {'input': 'Lavez-vous soigneusement les mains.', 'reference': 'Wash your hands carefully.'}, {'input': 'Nom de fantaisie Dosage', 'reference': 'Invented Name'}, {'input': 'Minervum 7061 NL-4817 ZK Breda Pays-Bas', 'reference': 'Minervum 7061 NL-4817 ZK Breda The Netherlands'}, {'input': 'Le darunavir a montré une activité antivirale synergique lors de son association avec les inhibiteurs de protéase tels que ritonavir, nelfinavir ou amprénavir et une activité antivirale additive lors de son association avec les inhibiteurs de protéase: indinavir, saquinavir, lopinavir, atazanavir ou tipranavir, avec les INTIs: zidovudine, lamivudine, zalcitabine, didanosine, stavudine, abacavir, emtricitabine ou ténofovir, avec les INNTIs: néviparine, delavirdine ou efavirenz, ainsi qu’avec l’inhibiteur de fusion enfuvirtide.', 'reference': 'Darunavir showed synergistic antiviral activity when studied in combination w

In [9]:
len(test_features)

102071

In [11]:
results = []
start_time = time.time()  # Start the timer

# Define a CSV file to save the results
output_csv_file = f"{model_name}_translation_results.csv"

# Write the header to the CSV file (run this once at the beginning)
with open(output_csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["input", "generated", "reference"])
    writer.writeheader()

def format_time(seconds):
    """Convert seconds into hours, minutes, and seconds."""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours}h {minutes}m {seconds}s"

for i, sample in enumerate(test_features):
    output = translation_pipeline(sample["input"], max_length=512)
    result = {
        "input": sample["input"],
        "generated": output[0]["translation_text"],
        "reference": sample["reference"]
    }
    results.append(result)

    # Append to CSV every 10 samples
    if i % 10 == 0:
        # Calculate elapsed time and ETA
        elapsed_time = time.time() - start_time
        avg_time_per_sample = elapsed_time / (i + 1)  # Avoid division by zero
        remaining_samples = len(test_features) - (i + 1)
        eta = avg_time_per_sample * remaining_samples

        # Clear the cell output (works in Jupyter Notebook)
        clear_output(wait=True)

        # Display progress
        print(f"Sample No.: {i}")
        print(f"Elapsed Time: {format_time(elapsed_time)}")
        print(f"ETA: {format_time(eta)}")
        print(f"Input: {sample['input']}")
        print(f"Generated: {output[0]['translation_text']}")
        print(f"Reference: {sample['reference']}\n")

        # Save the current results to the CSV file
        with open(output_csv_file, mode="a", newline="", encoding="utf-8") as file:
            writer = csv.DictWriter(file, fieldnames=["input", "generated", "reference"])
            writer.writerows(results)  # Append all new results
        results = []  # Clear the results list after saving

# Save any remaining results at the end
if results:
    with open(output_csv_file, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["input", "generated", "reference"])
        writer.writerows(results)

print(f"All results have been saved to {output_csv_file}")


Sample No.: 20
Elapsed Time: 0h 0m 13s
ETA: 18h 45m 21s
Input: • Règlement (CEE) n° 2309/93 du Conseil, du 22 juillet 1993 (JO L 214, 24.8.1993, p.
Generated:  Council Regulation (EEC) No 2309/93 of 22 July 1993 (OJ L 214, 24.8.1993, p.
Reference: • Council Regulation (EEC) No 2309/93 of 22 July 1993 (OJ L 214, 24.8.1993, p.



KeyboardInterrupt: 

In [12]:
results = pd.read_csv(output_csv_file)
results.head(2)

,input,generated,reference
0,Tel: +31 (0)182 557 755,Tel: +31 (0)182 557 755,Tel: +31 (0)182 557 755
1,5.3 Données de sécurité précliniques Les donné...,5.3 Pre-clinical safety data Preclinial data f...,5.3 Preclinical safety data Preclinical data r...


In [13]:
# BLEU score
results["generated"] = results["generated"].astype(str)
results["reference"] = results["reference"].astype(str)

# Extract the generated and reference translations
generated = results["generated"].tolist()
references = [results["reference"].tolist()]  # Wrap in a list as sacrebleu supports multiple references

# Calculate BLEU score
bleu = sacrebleu.corpus_bleu(generated, references)

print(f"BLEU score: {bleu.score}")

BLEU score: 41.69263275295573


In [18]:
# METEOR score
nltk.download('wordnet')

results["generated"] = results["generated"].astype(str)
results["reference"] = results["reference"].astype(str)

# Example DataFrame
data = {
    "hypothesis": results["generated"],
    "reference": results["reference"]
}
combined_results = pd.DataFrame(data)

# Tokenize the strings
combined_results["hypothesis"] = combined_results["hypothesis"].apply(lambda x: x.split())
combined_results["reference"] = combined_results["reference"].apply(lambda x: x.split())

# Calculate METEOR scores for each row
combined_results["meteor_score"] = combined_results.apply(lambda row: meteor_score([row["reference"]], row["hypothesis"]), axis=1)

overall_meteor_score = combined_results["meteor_score"].mean()

print(f"Overall METEOR score: {overall_meteor_score}")

Overall METEOR score: 0.5409643618058103


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
# BERT score
data = {
    "hypothesis": results["generated"],
    "reference": results["reference"]
}
combined_results = pd.DataFrame(data)

# Extract the hypothesis and reference as lists
hypotheses = combined_results["hypothesis"].tolist()
references = combined_results["reference"].tolist()

# Calculate BERTScore
P, R, F1 = score(hypotheses, references, lang="en", verbose=True)

# Compute overall BERTScore (mean of F1 scores)
overall_bert_score = sum(F1.tolist()) / len(F1)
print(f"Overall BERTScore: {overall_bert_score}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.60 seconds, 8.07 sentences/sec
Overall BERTScore: 0.9437672978355771
